In [3]:
import os
import re


def extract_basename(folder_path):
    tif_files = [f for f in os.listdir(folder_path) if f.endswith(".tif")]
    pattern = re.compile(r"^(.*?)(\d{4})\.tif$")
    for filename in tif_files:
        match = pattern.match(filename)
        if match:
            return match.group(1)
    return None


def check_all_image_files_present(folder_path, max_frame):
    basename = extract_basename(folder_path)
    if not basename:
        print("No valid .tif files found to extract basename.")
        return None
    missing_frames = []
    for frame_num in range(1, max_frame + 1):
        filename = f"{basename}{frame_num:04d}.tif"
        filepath = os.path.join(folder_path, filename)
        if not os.path.isfile(filepath):
            missing_frames.append(frame_num)
    if not missing_frames:
        return True
    else:
        return missing_frames


# # Example usage:
# if __name__ == "__main__":
#     folder = input("Enter folder path: ")
#     max_frame = int(input("Enter maximum frame number: "))
#     result = check_all_image_files_present(folder, max_frame)
#     if result is True:
#         print("true")
#     elif result is None:
#         print("Could not extract basename.")
#     else:
#         print("Missing frame numbers:", result)

In [6]:
from pathlib import Path
import pandas as pd

results = {}

dataPath = Path("../../data/KotaroData/")

mouse = "494_copy"
mousePath = dataPath / mouse

session_folders = sorted(mousePath.glob("Session*"))
for folder in session_folders:
    try:
        result = check_all_image_files_present(folder / "images", 9162)
        session_name = folder.name
        if result is True:
            results[session_name] = []
        elif result is None:
            results[session_name] = ["Could not extract basename."]
        else:
            results[session_name] = result
    except Exception as e:
        results[folder.name] = [f"Error: {e}"]

# Find the maximum number of missing frames for alignment
max_missing = max(len(v) for v in results.values())
data = {k: v + [""] * (max_missing - len(v)) for k, v in results.items()}

df = pd.DataFrame(data)
df.to_csv(dataPath / f"m{mouse}_missing_frames_report.csv", index=False)

No valid .tif files found to extract basename.
